 # Twitter API


In [6]:
import twitter 
import urlparse
from pprint import pprint as pp

In [5]:
class TwitterAPI(object):
    def __init__(self):
        consumer_key = "cAbLWtUerXPNmkyc0ITw0GYVP"
        consumer_secret ="8uXif247frgCiq5XnFgUiFb20KCloKC7aOnmLhCtkVSwKiZlQG"
        access_token = "277628349-bKVkguWOd2R6Y1cZe6YIvX96g7jpezy7pckQuQbt"
        access_secret = "9WaESyl7jkIYtRZg2NXr9gVKKGSFLpz3TB33tSNusBkA0"
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_secret = access_secret
        self.auth = twitter.oauth.OAuth(access_token, access_secret,
                                       consumer_key,consumer_secret)
        self.retries = 3
        self.a = twitter.Twitter(auth=self.auth)
    def searchTwitter(self, q, max_res = 10,**kwargs):
        search_results = self.a.search.tweets(q=q, count =10 , **kwargs)
        statuses = search_results['statuses']
        max_results = min(1000, max_res)
        for _ in range(10):
            try:
                next_results = search_results['search_metadata']['next_results']
            except keyerror as e:
                break
            next_results = urlparse.parse_qsl(next_results[1:])
            kwargs = dict(next_results)
            search_results = self.a.search.tweets(**kwargs)
            statuses += search_results['statuses']
            if len(statuses) >max_results:
                break
        return statuses
    def parseTweets(self, statuses):
        return [ (status['id'], status['created_at'],status['user']['id'],
                 status['user']['name'],status['text'],url['expanded_url'])
               for status in statuses
               for url in status['entities']['urls']]
    

In [41]:
t = TwitterAPI()
q = "ApacheSpark"
tsearch =t.searchTwitter(q)
pp(tsearch[1])

tparsed = t.parseTweets(tsearch)
pp(tparsed)

# Github API

In [1]:
from github import Github
from pprint import pprint as pp

In [2]:
ACCESS_TOKEN = "2bcc62867e3bf16f7acbabb2daccb6f6ba3e2b67"
USER = 'apache'
REPO = 'spark'
g = Github(ACCESS_TOKEN, per_page=100)

In [4]:
user = g.get_user(USER)
repo = g.get_repo(REPO)
len(repos_apache)
repos_apache = [repo.name for repo in g.get_user('apache').get_repos()]
pp(repo.get_languages())
stargazers = [s for s in repo.get_stargazers()]
print "Number of Stargazers " , len(stargazers)
[stargazers[i].login for i in range (0,20)]


# Meetup API

In [1]:
import json
import mimeparse
import urllib
import requests
from pprint import pprint as pp


In [8]:
MEETUP_API_HOST ='https://api.meetup.com'
EVENTS_URL=MEETUP_API_HOST+'/2/events.json'
MEMBERS_URL=MEETUP_API_HOST+'/2/members.json'
GROUP_URL=MEETUP_API_HOST+'/2/groups.json'
RSVPS_URL=MEETUP_API_HOST+'/2/rsvps.json'
PHOTOS_URL=MEETUP_API_HOST+'/2/photos.json'
GROUP_URLNAME='London-Machine-Learning-Meetup'

In [28]:
class MeetupAPI(object):
    def __init__(self,api_key, num_past_events = 10 , http_timeout = 5,
                http_retries = 2):
        self._api_key = api_key
        self._num_past_events = num_past_events 
        self._http_timeout = http_timeout
        self._http_retries = http_retries
    def get_past_events(self):
        params ={'key' :self._api_key,'group_url_name':GROUP_URL,
                'status':'past', 'desc' : 'true'}
        if self._num_past_events:
            params['page'] = str(self._num_past_events)
        query=urllib.urlencode(params)
        url='{0}?{1}'.format(EVENTS_URL,query)
        response=requests.get(url,timeout=self._http_timeout)
        data=response.json()['results']
        return data
    def get_members(self):
        params = {'key' : self._api_key, 'group_urlname':GROUP_URLNAME,
                  'offset' : '0','format' : 'json','page' : '100',
                  'order' : 'name'}
        query = urllib.urlencode(params)
        url = '{0}?{1}'.format(MEMBERS_URL, query)
        response = requests.get(url,timeout = self._http_timeout)
        data = response.json()['results']
        return data
    def get_groups_by_member(self):
        params = {'key' :  self._api_key,'member_id' : member_id,
                 'offset' : '0', 'format' :'json', 'page' : '100',
                 'order' : 'id'}
        query = urllib.urlencode(params)
        url = '{0}?{1}',format(GROUPS_URL, query)
        response = requests.get(url,timeout = self._http_timeout)
        data = response.json()['results']
        return data     
        
        

In [29]:
m = MeetupAPI(api_key = '6f4622777f6e14362a3b67332f437526')
last_meetups = m.get_past_events()
pp(last_meetups[5])
members = m.get_members()
pp(members[5])

# Yelp API

In [2]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator

auth = Oauth1Authenticator(
    consumer_key='NXN93OR84IV3um45SExHqw',
    consumer_secret='BHwSCrPgjPVINZOksaAHnX8Qv1U',
    token='Ti5UoUuOKVWeCiJuViWx8O-wJF05-UO3',
    token_secret='_8w3smiMiwOh-_muPdtnD3jHmlI'
)

client = Client(auth)

In [17]:
class YelpAPI(object):
    def __init__(self):
        consumer_key='NXN93OR84IV3um45SExHqw'
        consumer_secret='BHwSCrPgjPVINZOksaAHnX8Qv1U'
        token='Ti5UoUuOKVWeCiJuViWx8O-wJF05-UO3'
        token_secret='_8w3smiMiwOh-_muPdtnD3jHmlI'
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.token = token
        self.token_secret = token_secret
        self.auth = Oauth1Authenticator(consumer_key,consumer_secret,
                                   token,token_secret)
        self.client = Client(auth)
    def get_business(category,location):
        options =  'category=%s&location=%s&sort=1' % (category,location)
        url = 'http://api.yelp.com/v2/search?' + options




# CSV Data

In [8]:
import collections as c

In [9]:
class IO_csv(object):
    def __init__(self,filepath, filename, filesuffix = 'csv'):
        self.filepath = filepath
        self.filename = filename
        self.filesuffix = filesuffix
    def save(self, data, NTname, fields):
        NTuple = c.namedtuple(NTname, fields)
        if os.path.isfile('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                              self.filesuffix)):
            with open('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                           self.filesuffix),'ab') as f:
                writer = csv.writer(f)
                writer.writerow(fields)
                writer.writerows([row for row in map(NTuple._make,data)])
        else:
            with open('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                           self.filesuffix),'wb') as f:
                writer = csv.writer(f)
                writer.writerow(fields)
                writer.writerows([row for row in map(NTuple._make,data)])
    def load(self, NTname,fields):
        NTuple = c.namedtuple(NTname, fields)
        with open('{0}/{1}.{2}'.format(self.filepath, self.filename,
                                      self.filesuffix),'rU') as f:
            reader = csv.reader(f)
            for row in map(NTuple._make, reader):
                yield row
fields01 = ['id','created_at','user_id','user_name','tweet_text','url']
Tweet01 = c.namedtuple('Tweet01','fields')
def parse_tweet(data):
    return Tweet01(id = data.get('id',None),
                   created_at = data.get('created_at',None),
                   user_id = data.get('user_id',None),
                  user_name = data.get('user_name',None),
                  tweet_text = data.get('tweet_text',None),
                  url = data.get('url',None))

    

# JSON Data

In [10]:
class IO_json(object):
    def __init__(self,filepath, filename, filesuffix = 'csv'):
        self.filepath = filepath
        self.filename = filename
        self.filesuffix = filesuffix
    def save(self, data):
        if os.path.isfile('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                              self.filesuffix)):
            with io.open('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                           self.filesuffix),'a',
                        encoding = 'utf-8') as f:
                f.write(unicode(json.dumps(data,ensure_ascii = False)))
        else:
            with io.open('{0}/{1}.{2}'.format(self.filepath,self.filename,
                                           self.filesuffix),'w',
                        encoding = 'utf-8') as f:
                f.write(unicode(json.dumps(data,ensure_ascii = False)))
    def load(self):
        with open('{0}/{1}.{2}'.format(self.filepath, self.filename,
                                      self.filesuffix),encoding ='utf-8') as f:
            return f.read()


# Mongo DB

In [3]:
from pymongo import MongoClient as mc

In [4]:
class IO_Mongo(object):
    def __init__(self, db = 'twtr_db', coll = 'twtr_coll', **conn):
        self.client = mc(**conn)
        self.db = self.client[db]
        self.coll = self.client[coll]
    def save(self, data):
        return self.coll.insert(data)
    def load(self, return_cursor=False,criteria=None,projection=None):
        if criteria is None:
            criteria={}
        if projection is None:
            cursor=self.coll.find(criteria)
        else:
            cursor=self.coll.find(criteria,projection)
        if return_cursor:
            return cursor
        else:
            return[item for item in cursor]    